# Financial and Economic Data Applications
Original from Wes McKinney (https://github.com/wesm/pydata-book/tree/1st-edition)

Updated and edited for Python circa 2018 by R. D. Slater

Note: still need to fix Cells just before Quartile/Decile Analysis


In [1]:
from __future__ import division
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
pd.options.display.max_rows = 12
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 6))

In [2]:
%matplotlib inline

In [3]:
%pwd

'D:\\SMU\\qtw\\case2'

## Data munging topics

### Time series and cross-section alignment

In [13]:
close_px = pd.read_csv('D:\SMU\qtw\case2\stock_px.csv', parse_dates=True, index_col=0)
volume = pd.read_csv('D:/SMU/qtw/case2/volume.csv', parse_dates=True, index_col=0)
prices = close_px.loc['2011-09-05':'2011-09-14', ['AAPL', 'JNJ', 'SPX', 'XOM']]
volume = volume.loc['2011-09-05':'2011-09-12', ['AAPL', 'JNJ', 'XOM']]

In [14]:
prices

,AAPL,JNJ,SPX,XOM
2011-09-06,379.74,64.64,1165.24,71.15
2011-09-07,383.93,65.43,1198.62,73.65
2011-09-08,384.14,64.95,1185.90,72.82
2011-09-09,377.48,63.64,1154.23,71.01
2011-09-12,379.94,63.59,1162.27,71.84
2011-09-13,384.62,63.61,1172.87,71.65
2011-09-14,389.30,63.73,1188.68,72.64


In [15]:
volume

,AAPL,JNJ,XOM
2011-09-06,18173500,15848300,25416300
2011-09-07,12492000,10759700,23108400
2011-09-08,14839800,15551500,22434800
2011-09-09,20171900,17008200,27969100
2011-09-12,16697300,13448200,26205800


In [16]:
prices * volume

,AAPL,JNJ,SPX,XOM
2011-09-06,6.901205e+09,1.024434e+09,NaN,1.808370e+09
2011-09-07,4.796054e+09,7.040072e+08,NaN,1.701934e+09
2011-09-08,5.700561e+09,1.010070e+09,NaN,1.633702e+09
2011-09-09,7.614489e+09,1.082402e+09,NaN,1.986086e+09
2011-09-12,6.343972e+09,8.551710e+08,NaN,1.882625e+09
2011-09-13,NaN,NaN,NaN,NaN
2011-09-14,NaN,NaN,NaN,NaN


In [17]:
vwap = (prices * volume).sum() / volume.sum()

In [18]:
vwap

AAPL    380.655181
JNJ      64.394769
SPX            NaN
XOM      72.024288
dtype: float64

In [19]:
vwap.dropna()

AAPL    380.655181
JNJ      64.394769
XOM      72.024288
dtype: float64

In [20]:
prices.align(volume, join='inner')

(              AAPL    JNJ    XOM
 2011-09-06  379.74  64.64  71.15
 2011-09-07  383.93  65.43  73.65
 2011-09-08  384.14  64.95  72.82
 2011-09-09  377.48  63.64  71.01
 2011-09-12  379.94  63.59  71.84,                 AAPL       JNJ       XOM
 2011-09-06  18173500  15848300  25416300
 2011-09-07  12492000  10759700  23108400
 2011-09-08  14839800  15551500  22434800
 2011-09-09  20171900  17008200  27969100
 2011-09-12  16697300  13448200  26205800)

In [21]:
s1 = Series(range(3), index=['a', 'b', 'c'])
s2 = Series(range(4), index=['d', 'b', 'c', 'e'])
s3 = Series(range(3), index=['f', 'a', 'c'])
DataFrame({'one': s1, 'two': s2, 'three': s3})

,one,three,two
a,0.0,1.0,NaN
b,1.0,NaN,1.0
c,2.0,2.0,2.0
d,NaN,NaN,0.0
e,NaN,NaN,3.0
f,NaN,0.0,NaN


In [22]:
DataFrame({'one': s1, 'two': s2, 'three': s3}, index=list('face'))

,one,three,two
f,NaN,0.0,NaN
a,0.0,1.0,NaN
c,2.0,2.0,2.0
e,NaN,NaN,3.0


### Operations with time series of different frequencies

In [23]:
ts1 = Series(np.random.randn(3),
             index=pd.date_range('2012-6-13', periods=3, freq='W-WED'))
ts1

2012-06-13    1.631521
2012-06-20   -1.486790
2012-06-27    0.499050
Freq: W-WED, dtype: float64

In [24]:
ts1.resample('B')

DatetimeIndexResampler [freq=<BusinessDay>, axis=0, closed=left, label=left, convention=start, base=0]

In [25]:
ts1.resample('B').ffill()

2012-06-13    1.631521
2012-06-14    1.631521
2012-06-15    1.631521
2012-06-18    1.631521
2012-06-19    1.631521
2012-06-20   -1.486790
2012-06-21   -1.486790
2012-06-22   -1.486790
2012-06-25   -1.486790
2012-06-26   -1.486790
2012-06-27    0.499050
Freq: B, dtype: float64

In [26]:
dates = pd.DatetimeIndex(['2012-6-12', '2012-6-17', '2012-6-18',
                          '2012-6-21', '2012-6-22', '2012-6-29'])
ts2 = Series(np.random.randn(6), index=dates)
ts2

2012-06-12   -0.369016
2012-06-17   -1.359835
2012-06-18    0.097268
2012-06-21    0.449021
2012-06-22    1.223824
2012-06-29   -1.538004
dtype: float64

In [27]:
ts1.reindex(ts2.index, method='ffill')

2012-06-12         NaN
2012-06-17    1.631521
2012-06-18    1.631521
2012-06-21   -1.486790
2012-06-22   -1.486790
2012-06-29    0.499050
dtype: float64

In [28]:
ts2 + ts1.reindex(ts2.index, method='ffill')

2012-06-12         NaN
2012-06-17    0.271686
2012-06-18    1.728789
2012-06-21   -1.037769
2012-06-22   -0.262965
2012-06-29   -1.038954
dtype: float64

#### Using periods instead of timestamps

In [ ]:
gdp = Series([1.78, 1.94, 2.08, 2.01, 2.15, 2.31, 2.46],
             index=pd.period_range('1984Q2', periods=7, freq='Q-SEP'))
infl = Series([0.025, 0.045, 0.037, 0.04],
              index=pd.period_range('1982', periods=4, freq='A-DEC'))
gdp

In [ ]:
infl

In [ ]:
infl_q = infl.asfreq('Q-SEP', how='end')

In [ ]:
infl_q

In [ ]:
infl_q.reindex(gdp.index, method='ffill')

### Time of day and "as of" data selection

In [ ]:
# Make an intraday date range and time series
rng = pd.date_range('2012-06-01 09:30', '2012-06-01 15:59', freq='T')
# Make a 5-day series of 9:30-15:59 values
rng = rng.append([rng + pd.offsets.BDay(i) for i in range(1, 4)])
ts = Series(np.arange(len(rng), dtype=float), index=rng)
ts

In [ ]:
from datetime import time
ts[time(10, 0)]

In [ ]:
ts.at_time(time(10, 0))

In [ ]:
ts.between_time(time(10, 0), time(10, 1))

In [ ]:
np.random.seed(12346)

In [ ]:
# Set most of the time series randomly to NA
indexer = np.sort(np.random.permutation(len(ts))[700:])
irr_ts = ts.copy()
irr_ts[indexer] = np.nan
irr_ts['2012-06-01 09:50':'2012-06-01 10:00']

In [ ]:
selection = pd.date_range('2012-06-01 10:00', periods=4, freq='B')
irr_ts.asof(selection)

### Splicing together data sources

In [ ]:
data1 = DataFrame(np.ones((6, 3), dtype=float),
                  columns=['a', 'b', 'c'],
                  index=pd.date_range('6/12/2012', periods=6))
data2 = DataFrame(np.ones((6, 3), dtype=float) * 2,
                  columns=['a', 'b', 'c'],
                  index=pd.date_range('6/13/2012', periods=6))
spliced = pd.concat([data1.loc[:'2012-06-14'], data2.loc['2012-06-15':]])
spliced

In [ ]:
data2 = DataFrame(np.ones((6, 4), dtype=float) * 2,
                  columns=['a', 'b', 'c', 'd'],
                  index=pd.date_range('6/13/2012', periods=6))
spliced = pd.concat([data1.loc[:'2012-06-14'], data2.loc['2012-06-15':]])
spliced

In [ ]:
spliced_filled = spliced.combine_first(data2)
spliced_filled

In [ ]:
spliced.update(data2, overwrite=False)

In [ ]:
spliced

In [ ]:
cp_spliced = spliced.copy()
cp_spliced[['a', 'c']] = data1[['a', 'c']]
cp_spliced

### Return indexes and cumulative returns

In [ ]:
from pandas_datareader import data as web
price = web.get_data_yahoo('AAPL', '2011-01-01')['Adj Close']
price[-5:]

In [ ]:
price['2011-10-03'] / price['2011-3-01'] - 1

In [ ]:
returns = price.pct_change()
ret_index = (1 + returns).cumprod()
ret_index[0] = 1  # Set first value to 1
ret_index

In [ ]:
m_returns = ret_index.resample('BM').last().pct_change()
m_returns['2012']

In [ ]:
m_rets = (1 + returns).resample('M', kind='period').prod() - 1
m_rets['2012']

## Group transforms and analysis

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 10
np.random.seed(12345)

In [ ]:
import random; random.seed(0)
import string

N = 1000
def rands(n):
    choices = string.ascii_uppercase
    return ''.join([random.choice(choices) for _ in range(n)])
tickers = np.array([rands(5) for _ in range(N)])

In [ ]:
M = 500
df = DataFrame({'Momentum' : np.random.randn(M) / 200 + 0.03,
                'Value' : np.random.randn(M) / 200 + 0.08,
                'ShortInterest' : np.random.randn(M) / 200 - 0.02},
                index=tickers[:M])

In [ ]:
ind_names = np.array(['FINANCIAL', 'TECH'])
sampler = np.random.randint(0, len(ind_names), N)
industries = Series(ind_names[sampler], index=tickers,
                    name='industry')

In [ ]:
by_industry = df.groupby(industries)
by_industry.mean()

In [ ]:
by_industry.describe()

In [ ]:
# Within-Industry Standardize
def zscore(group):
    return (group - group.mean()) / group.std()

df_stand = by_industry.apply(zscore)

In [ ]:
df_stand.groupby(industries).agg(['mean', 'std'])

In [ ]:
# Within-industry rank descending
ind_rank = by_industry.rank(ascending=False)
ind_rank.groupby(industries).agg(['min', 'max'])

In [ ]:
# Industry rank and standardize
by_industry.apply(lambda x: zscore(x.rank()))

### Group factor exposures

In [ ]:
from numpy.random import rand
fac1, fac2, fac3 = np.random.rand(3, 1000)

ticker_subset = tickers.take(np.random.permutation(N)[:1000])

# Weighted sum of factors plus noise
port = Series(0.7 * fac1 - 1.2 * fac2 + 0.3 * fac3 + rand(1000),
              index=ticker_subset)
factors = DataFrame({'f1': fac1, 'f2': fac2, 'f3': fac3},
                    index=ticker_subset)

In [ ]:
factors.corrwith(port)

In [ ]:
from pyfinance.ols import PandasRollingOLS
model=PandasRollingOLS(y=port, x=factors,window=10)
model.beta

In [ ]:
def beta_exposure(chunk, factors=None):
    return feature1(y=chunk, x=factors).beta

In [ ]:
by_ind = port.groupby(industries)
exposures = by_ind.apply(beta_exposure, factors=factors)
exposures.unstack()

### Decile and quartile analysis

In [ ]:
import pandas_datareader.data as web
data = web.get_data_yahoo('SPY', '2006-01-01')
data.info()

In [ ]:
px = data['Adj Close']
returns = px.pct_change()

def to_index(rets):
    index = (1 + rets).cumprod()
    first_loc = max(index.index.get_loc(index.idxmax()) - 1, 0)
    index.values[first_loc] = 1
    return index

def trend_signal(rets, lookback, lag):
    signal = pd.rolling_sum(rets, lookback, min_periods=lookback - 5)
    return signal.shift(lag)

In [ ]:
signal = trend_signal(returns, 100, 3)
trade_friday = signal.resample('W-FRI').resample('B').ffill()
trade_rets = trade_friday.shift(1) * returns
trade_rets = trade_rets[:len(returns)]

In [ ]:
to_index(trade_rets).plot()

In [ ]:
vol = pd.rolling_std(returns, 250, min_periods=200) * np.sqrt(250)

def sharpe(rets, ann=250):
    return rets.mean() / rets.std()  * np.sqrt(ann)

In [ ]:
cats = pd.qcut(vol, 4)
print('cats: %d, trade_rets: %d, vol: %d' % (len(cats), len(trade_rets), len(vol)))

In [ ]:
trade_rets.groupby(cats).agg(sharpe)

## More example applications

### Signal frontier analysis

In [ ]:
import datetime
import pandas as pd
start_dt=datetime.datetime(2009,1,1)
end_dt=datetime.datetime(2012,6,1)
from pandas_datareader import data as web
stock='AAPL'
px=pd.DataFrame({'AAPL':web.get_data_yahoo(stock, start_dt, end_dt)['Adj Close']})
names=['GOOG','MSFT', 'GS','INTC', 'MS', 'BAC', 'C']
for stock in names:
    while True:
        try:
            px[stock]=web.get_data_yahoo(stock, start_dt, end_dt)['Adj Close']
            break
        except:
            print('Unable to read stock: {0}'.format(stock))
            print('trying again')


In [ ]:
#px = pd.read_csv('ch11/stock_px.csv')

In [ ]:
plt.close('all')

In [ ]:
px = px.asfreq('B').fillna(method='pad')
rets = px.pct_change()
((1 + rets).cumprod() - 1).plot()

In [ ]:
def calc_mom(price, lookback, lag):
    mom_ret = price.shift(lag).pct_change(lookback)
    ranks = mom_ret.rank(axis=1, ascending=False)
    demeaned = ranks.subtract(ranks.mean(axis=1), axis=0)
    return demeaned.divide(demeaned.std(axis=1), axis=0)

In [ ]:
compound = lambda x : (1 + x).prod() - 1
daily_sr = lambda x: x.mean() / x.std()

def strat_sr(prices, lb, hold):
    # Compute portfolio weights
    freq = '%dB' % hold
    port = calc_mom(prices, lb, lag=1)

    daily_rets = prices.pct_change()

    # Compute portfolio returns
    port = port.shift(1).resample(freq, how='first')
    returns = daily_rets.resample(freq, how=compound)
    port_rets = (port * returns).sum(axis=1)

    return daily_sr(port_rets) * np.sqrt(252 / hold)

In [ ]:
strat_sr(px, 70, 30)

In [ ]:
from collections import defaultdict

lookbacks = range(20, 90, 5)
holdings = range(20, 90, 5)
dd = defaultdict(dict)
for lb in lookbacks:
    for hold in holdings:
        dd[lb][hold] = strat_sr(px, lb, hold)

ddf = DataFrame(dd)
ddf.index.name = 'Holding Period'
ddf.columns.name = 'Lookback Period'

In [ ]:
import matplotlib.pyplot as plt

def heatmap(df, cmap=plt.cm.gray_r):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    axim = ax.imshow(df.values, cmap=cmap, interpolation='nearest')
    ax.set_xlabel(df.columns.name)
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_xticklabels(list(df.columns))
    ax.set_ylabel(df.index.name)
    ax.set_yticks(np.arange(len(df.index)))
    ax.set_yticklabels(list(df.index))
    plt.colorbar(axim)

In [ ]:
heatmap(ddf)
plt.show()

### Future contract rolling

In [29]:
pd.options.display.max_rows = 10

In [42]:
#import pandas.io.data as web
# Approximate price of S&P 500 index
px=pd.read_csv('D:\SMU\qtw\case2\case_study.csv',index_col='Date')

px

,IBM,TXN,CSCO,QCOM,ORCL,NOK,AMAT
Date,,,,,,,
2009-01-02,69.907677,12.922787,13.908728,30.000746,16.574270,10.815960,8.829226
2009-01-05,69.467598,12.777762,14.031741,29.514910,16.286180,10.775351,8.829226
2009-01-06,71.395920,13.253106,14.589402,30.081724,16.556271,10.971633,9.582232
2009-01-07,70.243721,12.552179,14.203962,28.786135,15.890058,10.599371,8.771300
2009-01-08,69.755669,12.310483,14.384381,28.567513,15.863053,10.403084,8.887151
...,...,...,...,...,...,...,...
2012-05-25,165.667526,24.999376,13.640099,48.970528,24.172287,2.377202,9.465686
2012-05-29,167.509232,25.362183,13.857271,49.884659,24.468197,2.486789,9.618358
2012-05-30,165.863632,25.008015,13.690218,49.292641,24.200031,2.309764,9.393839


In [44]:
from datetime import datetime
expiry = {'ESU2': datetime(2012, 9, 21),
          'ESZ2': datetime(2012, 12, 21)}
expiry = Series(expiry).sort_values()
px.values

array([[  69.9077,   12.9228,   13.9087, ...,   16.5743,   10.816 ,
           8.8292],
       [  69.4676,   12.7778,   14.0317, ...,   16.2862,   10.7754,
           8.8292],
       [  71.3959,   13.2531,   14.5894, ...,   16.5563,   10.9716,
           9.5822],
       ..., 
       [ 165.8636,   25.008 ,   13.6902, ...,   24.2   ,    2.3098,
           9.3938],
       [ 164.4738,   24.602 ,   13.6401, ...,   24.4774,    2.2508,
           9.2771],
       [ 161.2168,   23.3235,   13.331 , ...,   24.0428,    2.2255,
           8.9897]])

In [39]:
expiry

ESU2   2012-09-21
ESZ2   2012-12-21
dtype: datetime64[ns]

In [41]:
np.random.seed(12347)
N = 200
walk = (np.random.randint(0, 200, size=N) - 100) * 0.25
perturb = (np.random.randint(0, 20, size=N) - 10) * 0.25
walk = walk.cumsum()

rng = pd.date_range(px.index[0], periods=len(px) + N, freq='B')
near = np.concatenate([px.values, px.values[-1] + walk])
far = np.concatenate([px.values, px.values[-1] + walk + perturb])
prices = DataFrame({'ESU2': near, 'ESZ2': far}, index=rng)

ValueError: operands could not be broadcast together with shapes (8,) (200,) 

In [36]:
prices.tail()

,AAPL,JNJ,SPX,XOM
2011-09-08,384.14,64.95,1185.90,72.82
2011-09-09,377.48,63.64,1154.23,71.01
2011-09-12,379.94,63.59,1162.27,71.84
2011-09-13,384.62,63.61,1172.87,71.65
2011-09-14,389.30,63.73,1188.68,72.64


In [45]:
def get_roll_weights(start, expiry, items, roll_periods=5):
    # start : first date to compute weighting DataFrame
    # expiry : Series of ticker -> expiration dates
    # items : sequence of contract names

    dates = pd.date_range(start, expiry[-1], freq='B')
    weights = DataFrame(np.zeros((len(dates), len(items))),
                        index=dates, columns=items)

    prev_date = weights.index[0]
    for i, (item, ex_date) in enumerate(expiry.iteritems()):
        if i < len(expiry) - 1:
            weights.ix[prev_date:ex_date - pd.offsets.BDay(), item] = 1
            roll_rng = pd.date_range(end=ex_date - pd.offsets.BDay(),
                                     periods=roll_periods + 1, freq='B')

            decay_weights = np.linspace(0, 1, roll_periods + 1)
            weights.ix[roll_rng, item] = 1 - decay_weights
            weights.ix[roll_rng, expiry.index[i + 1]] = decay_weights
        else:
            weights.ix[prev_date:, item] = 1

        prev_date = ex_date

    return weights

In [46]:
weights = get_roll_weights('6/1/2012', expiry, prices.columns)
weights.loc['2012-09-12':'2012-09-21']

C:\Users\rlagr\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


,AAPL,JNJ,SPX,XOM,ESU2,ESZ2
2012-09-12,0.0,0.0,0.0,0.0,1.0,NaN
2012-09-13,0.0,0.0,0.0,0.0,1.0,0.0
2012-09-14,0.0,0.0,0.0,0.0,0.8,0.2
2012-09-17,0.0,0.0,0.0,0.0,0.6,0.4
2012-09-18,0.0,0.0,0.0,0.0,0.4,0.6
2012-09-19,0.0,0.0,0.0,0.0,0.2,0.8
2012-09-20,0.0,0.0,0.0,0.0,0.0,1.0
2012-09-21,0.0,0.0,0.0,0.0,NaN,1.0


In [47]:
rolled_returns = (prices.pct_change() * weights).sum(1)

### Rolling correlation and linear regression

In [48]:
aapl = web.get_data_yahoo('AAPL', '2000-01-01')['Adj Close']
msft = web.get_data_yahoo('MSFT', '2000-01-01')['Adj Close']

aapl_rets = aapl.pct_change()
msft_rets = msft.pct_change()

NameError: name 'web' is not defined

In [ ]:
plt.figure()


In [ ]:
pd.rolling_corr(aapl_rets, msft_rets, 250).plot()

In [ ]:
plt.figure()

In [ ]:
from pyfinance.ols import PandasRollingOLS
model = PandasRollingOLS(y=aapl_rets, x= msft_rets, window=250)
model.beta

In [ ]:
model.beta['feature1'].plot()

In [ ]:
msft_rets